# Exploring Hacker News Post
---
In this project we are exploring datest containing sampled 20k+ hacker news post.  It has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. The dataset can be found on [Kaggle](https://www.kaggle.com/hacker-news/hacker-news-posts)